In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Function

In [2]:
def an_filename(name):
    '''Get the real file name not the path from csv file'''
    filename = "a_" + name.split("a_")[1]
    return filename

def sen_filename(name):
    '''Get the real file name not the path from csv file'''
    filename = "s_" + name.split("s_")[1]
    return filename

In [3]:
def convert(seconds): 
    milli = (seconds % 1) * 100
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
      
    return "%d:%02d:%02d:%02d" % (hour, minutes, seconds, milli)

In [4]:
def key(X):
    key, value = list(X.items())[0]
    return key.strip("()")

def value(X):
    key, value = list(X.items())[0]
    key2, value2 = list(value[0].items())[0]
    return value2

def process(X):
    key, value = list(X.items())[0]
    key2, value2 = list(value[0].items())[0]
    return key2

In [5]:
def check_between(time_stamp):
    for i in range(fall_anclean1.shape[0]):
        if fall_anclean1.loc[i,"start"] < time_stamp and time_stamp < fall_anclean1.loc[i,"end"]:
            return fall_anclean1.loc[i,"activitiy"] + " " + str(fall_anclean1.loc[i,"batch"])   
    return None

### Prepare Annotation dataframe

In [ ]:
anno_basepath = "../raw_data/full_data/" # using for iterate path
anno_csvpath = "../raw_data/sample/annotations.csv"
an_csv = pd.read_csv(anno_csvpath)
an_csv["real_filename"] = an_csv["filename"].apply(an_filename)
an_csv # make real file name for iterate path

In [ ]:
an_df = pd.read_json("../raw_data/sample/annot/" + an_csv["real_filename"][0])

In [ ]:
an_df["activities"] = an_df["dt"].apply(key)
an_df["process"] = an_df["dt"].apply(process)
an_df["time"] = an_df["dt"].apply(value)
an_df = an_df[an_df.columns[-3:]]

In [ ]:
row = an_df.shape[0]
n = 0
list_begin = []
list_end = []
list_act = []
for row in range(int(row/2)):
    list_begin.append(an_df["time"][n])
    list_act.append(an_df["activities"][n])
    list_end.append(an_df["time"][n+1])
    n +=2

In [ ]:
an_dict_clean = {"activitiy" : list_act, "start" : list_begin, "end" : list_end}
an_df_clean = pd.DataFrame(an_dict_clean)

In [ ]:
an_df_clean.head()

In [ ]:
batch = []
batch_no = 0
start = []
end = []
activitiy = []
for row in range(an_df_clean.shape[0]):
    start_an = an_df_clean.loc[row, "start"]
    end_an = start_an + 2000
    if end_an > an_df_clean.loc[row, "end"]:
        continue
    start.append(start_an)
    end.append(end_an)
    activitiy.append(an_df_clean.loc[row, "activitiy"])
    batch.append(batch_no)
    batch_no += 1
    start_an += 2000

In [ ]:
an_dict_clean = {"activitiy" : activitiy, "start" : start, "end" : end, "batch" : batch}
an_df_clean = pd.DataFrame(an_dict_clean)

In [ ]:
an_df_clean.head()

### Prepare Sensor dataframe

In [ ]:
sensor_csvpath = "../raw_data/sample/sensors.csv"
sen_csv = pd.read_csv(sensor_csvpath)
sen_csv["real_filename"] = sen_csv["filename"].apply(sen_filename)
sen_csv.head()

In [ ]:
sen_path_base = "../raw_data/sample/sens/"

In [ ]:
sample_path = sen_path_base + sen_csv["real_filename"][50];sample_path

In [ ]:
sen_json = pd.read_json(sample_path)
sen_json.head()

In [ ]:
sen_acc = np.array(sen_json.dt["a"])
acc_df = pd.DataFrame(sen_acc, columns=["time_stamp", "sensor_time_stamp","X","Y","Z"])

In [ ]:
def check_between(time_stamp):
    for i in range(an_df_clean.shape[0]):
        if an_df_clean.loc[i,"start"] < time_stamp and time_stamp < an_df_clean.loc[i,"end"]:
            return an_df_clean.loc[i,"activitiy"] + " " + str(an_df_clean.loc[i,"batch"])   
    return None

In [ ]:
#All file in the folder
all_jason_df = pd.DataFrame(columns=["time_stamp","sensor_time_stamp","X","Y","Z"])
for sen_jsonpath in sen_csv["real_filename"]:
    sensor_path = sen_path_base + sen_jsonpath
    sen_json = pd.read_json(sensor_path)
    if sen_json.dt["a"] == []:
        continue
    sen_acc = np.array(sen_json.dt["a"])
    acc_df = pd.DataFrame(sen_acc, columns=["time_stamp", "sensor_time_stamp","X","Y","Z"])
    all_jason_df = pd.concat([all_jason_df,acc_df])

In [ ]:
all_jason_df["activity"] = all_jason_df["time_stamp"].apply(check_between)

In [ ]:
all_jason_df.isnull().sum()

In [ ]:
all_jason_df.drop("sensor_time_stamp",axis = 1,inplace=True)
all_jason_df.to_csv("sample.csv", index=False)

### Fall data

In [6]:
fall_path_csv1 = "../raw_data/20201130-sg9-y-fall/a_2020-11-30-W6Ex5DZKnuMe8eiFIsbSutgVLV72.json"
fall_an_1 = pd.read_json(fall_path_csv1)
fall_an_1.head()

,dt,os,p,pf,uid,v
0,{'walk': [{'Begin': 1606746577754}]},29,SM-G960F,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1
1,{'walk': [{'End': 1606746581344}]},29,SM-G960F,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1
2,{'fall': [{'Begin': 1606746581354}]},29,SM-G960F,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1
3,{'fall': [{'End': 1606746582904}]},29,SM-G960F,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1
4,{'fall': [{'Begin': 1606746600253}]},29,SM-G960F,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1


In [7]:
fall_an_1["activities"] = fall_an_1["dt"].apply(key)
fall_an_1["process"] = fall_an_1["dt"].apply(process)
fall_an_1["time"] = fall_an_1["dt"].apply(value)
fall_an_1 = fall_an_1[fall_an_1.columns[-3:]]

In [8]:
row = fall_an_1.shape[0]
n = 0
list_begin = []
list_end = []
list_act = []
for row in range(int(row/2)):
    list_begin.append(fall_an_1["time"][n])
    list_act.append(fall_an_1["activities"][n])
    list_end.append(fall_an_1["time"][n+1])
    n +=2

In [9]:
fall_an_dict1 = {"activitiy" : list_act, "start" : list_begin, "end" : list_end}
fall_an_1 = pd.DataFrame(fall_an_dict1)

In [10]:
fall_an_1["interval"] = (fall_an_1["end"] - fall_an_1["start"])//2000

In [ ]:
fall_an_1

In [11]:
batch = []
batch_no = 0
start = []
end = []
activitiy = []
for row in range(fall_an_1.shape[0]):
    start_an = fall_an_1.loc[row, "start"]
    end_an = start_an + 2000
    while end_an < fall_an_1.loc[row, "end"]:
        start.append(start_an)
        end.append(end_an)
        activitiy.append(fall_an_1.loc[row, "activitiy"])
        batch.append(batch_no)
        batch_no += 1
        start_an += 2000
        end_an += 2000
    if fall_an_1.loc[row, "activitiy"] == "fall":
        start.append(start_an)
        end.append(fall_an_1.loc[row, "end"])
        activitiy.append(fall_an_1.loc[row, "activitiy"])
        batch.append(batch_no)
        batch_no += 1

In [12]:
fall_an_dict1 = {"activitiy" : activitiy, "start" : start, "end" : end, "batch" : batch}
fall_anclean1 = pd.DataFrame(fall_an_dict1)

In [13]:
fall_anclean1.head()

,activitiy,start,end,batch
0,walk,1606746577754,1606746579754,0
1,fall,1606746581354,1606746582904,1
2,fall,1606746600253,1606746602253,2
3,fall,1606746602253,1606746602547,3
4,walk,1606746611805,1606746613805,4


In [14]:
import os
list_json = os.listdir('../raw_data/20201130-sg9-y-fall/')
list_json = [ s for s in list_json if "s_" in s]

In [15]:
jsonpath = "../raw_data/20201130-sg9-y-fall/"

In [16]:
all_jason_df = pd.DataFrame(columns=["time_stamp","sensor_time_stamp","X","Y","Z"])
for sen_jsonpath in list_json:
    sensor_path = jsonpath + sen_jsonpath
    sen_json = pd.read_json(sensor_path)
    if sen_json.dt["a"] == []:
        continue
    sen_acc = np.array(sen_json.dt["a"])
    acc_df = pd.DataFrame(sen_acc, columns=["time_stamp", "sensor_time_stamp","X","Y","Z"])
    all_jason_df = pd.concat([all_jason_df,acc_df])

In [17]:
all_jason_df["activity"] = all_jason_df["time_stamp"].apply(check_between)

In [18]:
all_jason_df.dropna(inplace=True)

In [19]:
all_jason_df["model"] = "SM_G960F"
all_jason_df.reset_index(inplace=True)


In [20]:
all_jason_df.drop("index", axis = 1, inplace=True)

In [21]:
all_jason_df.head()

,time_stamp,sensor_time_stamp,X,Y,Z,activity,model
0,1.606747e+12,4.298059e+13,3.119740,6.373559,5.188394,walk 0,SM_G960F
1,1.606747e+12,4.298061e+13,2.650462,6.399897,5.490072,walk 0,SM_G960F
2,1.606747e+12,4.298063e+13,2.214704,6.428628,6.732701,walk 0,SM_G960F
3,1.606747e+12,4.298065e+13,1.920208,6.079064,7.604217,walk 0,SM_G960F
4,1.606747e+12,4.298067e+13,1.996825,6.086246,7.324087,walk 0,SM_G960F


In [22]:
test_row = all_jason_df.loc[0,"time_stamp":"Z"]
test_row2 = all_jason_df.loc[1,"time_stamp":"Z"]

In [23]:
# test_row = test_row.astype('float64', raise_on_error = False)
test_row = test_row.to_numpy(dtype = 'float64')
test_row2 = test_row2.to_numpy(dtype = 'float64')

In [24]:
test3 = np.append([test_row],[test_row2], axis = 0)
np.append(test3,[test_row2], axis = 0)

array([[1.60674658e+12, 4.29805931e+13, 3.11974001e+00, 6.37355947e+00,
        5.18839359e+00],
       [1.60674658e+12, 4.29806131e+13, 2.65046215e+00, 6.39989662e+00,
        5.49007225e+00],
       [1.60674658e+12, 4.29806131e+13, 2.65046215e+00, 6.39989662e+00,
        5.49007225e+00]])

In [25]:
npypath = "../raw_data/fall_clean/"

In [27]:
n = 0
for row in range(all_jason_df.shape[0]):
    curr_act = all_jason_df.loc[row,"activity"]
    curr_act = curr_act.split()[0]
    curr_batch = all_jason_df.loc[row,"activity"]
    curr_batch = curr_batch.split()[1]
    model = all_jason_df.loc[row,"model"]
    curr_time = int(all_jason_df.loc[row,"time_stamp"])
    acc_value = all_jason_df.loc[row,"time_stamp":"Z"]
    acc_value = [acc_value.to_numpy(dtype = 'float64')]
    if row == 0:
        acc = acc_value
        pre_batch = curr_batch
    else:
        if pre_batch == curr_batch:
            acc = np.append(acc,acc_value, axis = 0)
            pre_batch = curr_batch
            n += 1
            save_path = f"{npypath}/{curr_act}/{curr_act}_{model}_{curr_time}.npy"
        else:
            if n == 0:
                save_path = f"{npypath}/{curr_act}/{curr_act}_{model}_{curr_time}.npy"
            zero = np.zeros( acc.shape, dtype=float)
            stack_gb = np.stack((acc,zero,zero))
            np.save(save_path, stack_gb)
            pre_batch = curr_batch
            acc = acc_value

### Fall data2

In [28]:
fall_path_csv1 = "../raw_data/2020-12-02-fall-only/annot/a_2020-12-02-W6Ex5DZKnuMe8eiFIsbSutgVLV72.json"
fall_an_1 = pd.read_json(fall_path_csv1)

fall_an_1["activities"] = fall_an_1["dt"].apply(key)
fall_an_1["process"] = fall_an_1["dt"].apply(process)
fall_an_1["time"] = fall_an_1["dt"].apply(value)
fall_an_1 = fall_an_1[fall_an_1.columns[-3:]]

row = fall_an_1.shape[0]
n = 0
list_begin = []
list_end = []
list_act = []
for row in range(int(row/2)):
    list_begin.append(fall_an_1["time"][n])
    list_act.append(fall_an_1["activities"][n])
    list_end.append(fall_an_1["time"][n+1])
    n +=2
    
fall_an_dict1 = {"activitiy" : list_act, "start" : list_begin, "end" : list_end}
fall_an_1 = pd.DataFrame(fall_an_dict1)

In [29]:
batch = []
batch_no = 0
start = []
end = []
activitiy = []
for row in range(fall_an_1.shape[0]):
    start_an = fall_an_1.loc[row, "start"]
    end_an = start_an + 2000
    while end_an < fall_an_1.loc[row, "end"]:
        start.append(start_an)
        end.append(end_an)
        activitiy.append(fall_an_1.loc[row, "activitiy"])
        batch.append(batch_no)
        batch_no += 1
        start_an += 2000
        end_an += 2000
    if fall_an_1.loc[row, "activitiy"] == "fall":
        start.append(start_an)
        end.append(fall_an_1.loc[row, "end"])
        activitiy.append(fall_an_1.loc[row, "activitiy"])
        batch.append(batch_no)
        batch_no += 1

In [30]:
fall_an_dict1 = {"activitiy" : activitiy, "start" : start, "end" : end, "batch" : batch}
fall_anclean1 = pd.DataFrame(fall_an_dict1)

In [31]:
fall_anclean1.head()

,activitiy,start,end,batch
0,walk,1606911580361,1606911582361,0
1,walk,1606911582361,1606911584361,1
2,walk,1606911584361,1606911586361,2
3,fall,1606911587439,1606911589439,3
4,fall,1606911589439,1606911590411,4


In [32]:
import os
jsonpath = "../raw_data/2020-12-02-fall-only/sens/"
list_json = os.listdir(jsonpath)
list_json = [ s for s in list_json if "s_" in s]

all_jason_df = pd.DataFrame(columns=["time_stamp","sensor_time_stamp","X","Y","Z"])
for sen_jsonpath in list_json:
    sensor_path = jsonpath + sen_jsonpath
    sen_json = pd.read_json(sensor_path)
    if sen_json.dt["a"] == []:
        continue
    sen_acc = np.array(sen_json.dt["a"])
    acc_df = pd.DataFrame(sen_acc, columns=["time_stamp", "sensor_time_stamp","X","Y","Z"])
    all_jason_df = pd.concat([all_jason_df,acc_df])

In [33]:
all_jason_df["activity"] = all_jason_df["time_stamp"].apply(check_between)


In [34]:
all_jason_df.dropna(inplace=True)
all_jason_df["model"] = "ASUS_X013DB"
all_jason_df.reset_index(inplace=True)
all_jason_df.drop("index", axis = 1, inplace=True)

In [35]:
all_jason_df.head()

,time_stamp,sensor_time_stamp,X,Y,Z,activity,model
0,1.606912e+12,5.520928e+11,2.219086,10.452423,0.565140,fall 21,ASUS_X013DB
1,1.606912e+12,5.521130e+11,3.239151,15.479416,-4.748779,fall 21,ASUS_X013DB
2,1.606912e+12,5.521331e+11,2.286484,16.656570,-7.636368,fall 21,ASUS_X013DB
3,1.606912e+12,5.521532e+11,-0.952286,12.195251,-6.519180,fall 21,ASUS_X013DB
4,1.606912e+12,5.521734e+11,-2.065643,7.709763,-2.027466,fall 21,ASUS_X013DB


In [36]:
n = 0
npypath = "../raw_data/fall_clean/"
for row in range(all_jason_df.shape[0]):
    curr_act = all_jason_df.loc[row,"activity"]
    curr_act = curr_act.split()[0]
    curr_batch = all_jason_df.loc[row,"activity"]
    curr_batch = curr_batch.split()[1]
    model = all_jason_df.loc[row,"model"]
    curr_time = int(all_jason_df.loc[row,"time_stamp"])
    acc_value = all_jason_df.loc[row,"time_stamp":"Z"]
    acc_value = [acc_value.to_numpy(dtype = 'float64')]
    if row == 0:
        acc = acc_value
        pre_batch = curr_batch
    else:
        if pre_batch == curr_batch:
            acc = np.append(acc,acc_value, axis = 0)
            pre_batch = curr_batch
            n += 1
            save_path = f"{npypath}/{curr_act}/{curr_act}_{model}_{curr_time}.npy"
        else:
            if n == 0:
                save_path = f"{npypath}/{curr_act}/{curr_act}_{model}_{curr_time}.npy"
            zero = np.zeros( acc.shape, dtype=float)
            stack_gb = np.stack((acc,zero,zero))
            np.save(save_path, stack_gb)
            pre_batch = curr_batch
            acc = acc_value